In [ ]:
from arcgis.gis import GIS
from arcgis.raster.functions import apply

In [ ]:
gis = GIS('https://ionutsandric.maps.arcgis.com', 'ionutsandric', 'icub2018')

In [ ]:
landsat_item = gis.content.search('Multispectral Landsat', 'Imagery Layer', outside_org=True)[0]

In [ ]:
landsat_item

In [ ]:
sentinel_item = gis.content.search('Multispectral Sentinel-2', 'Imagery Layer', outside_org=True)[0]

In [ ]:
sentinel_item

In [ ]:
# View imagery properties
from IPython.display import HTML

In [ ]:
HTML(landsat_item.description)

In [ ]:
HTML(sentinel_item.description)

In [ ]:
# Access and use layers from Image Services
landsat = landsat_item.layers[0]
landsat

In [ ]:
sentinel = sentinel_item.layers[0]
sentinel

In [ ]:
# Explore wavelengths
import pandas as pd

In [ ]:
pd.DataFrame(landsat.key_properties()['BandProperties'])

In [ ]:
pd.DataFrame(sentinel.key_properties()['BandProperties'])

In [ ]:
# Vizualize layers in map

In [ ]:
m = gis.map('Bucharest, Romania')
m

In [ ]:
for rasterfunc in landsat.properties.rasterFunctionInfos:
    print(rasterfunc.name)

In [ ]:
for rasterfunc in sentinel.properties.rasterFunctionInfos:
    print(rasterfunc.name)

In [ ]:
# Raster function for NDVI
from arcgis.geocoding import geocode
area = geocode('Bucharest, Romania', out_sr=landsat.properties.spatialReference)[0]

In [ ]:
landsat.extent = area['extent']
landsat

In [ ]:
sentinel.extent = area['extent']
sentinel

In [ ]:
ndvi_colorized = apply(landsat, 'NDVI Colorized')
ndvi_colorized

In [ ]:
ndvi_colorized_sentinel = apply(sentinel, 'Normalized Difference Built-Up Index (NDBI)')
ndvi_colorized_sentinel

In [ ]:
# Custom bands

In [ ]:
from arcgis.raster.functions import stretch, extract_band

In [ ]:
naturalcolor = stretch(extract_band(landsat, [3,2,1]), 
                    stretch_type='percentclip', min_percent=0.1, max_percent=0.1, gamma=[1, 1, 1], dra=True)

In [ ]:
naturalcolor

In [ ]:
naturalcolor_sentinel = stretch(extract_band(sentinel, [3,2,1]), 
                    stretch_type='percentclip', min_percent=0.1, max_percent=0.1, gamma=[1, 1, 1], dra=True)

In [ ]:
naturalcolor_sentinel

In [ ]:
# Spectral profile from the sampled values at a location

In [ ]:
m_sprofile = gis.map('Bucharest, Romania')
m_sprofile.basemap = 'topo-vector'
m_sprofile.add_layer(landsat)
m_sprofile

In [ ]:
from bokeh.models import Range1d
from bokeh.plotting import figure, show, output_notebook
from IPython.display import clear_output
output_notebook()

def get_samples(mw, g):
    clear_output()
    m_sprofile.draw(g)
    samples = landsat.get_samples(g, pixel_size=30)
    values = samples[0]['value']
    vals = [float(int(s)/100000) for s in values.split(' ')]
    
    x = ['1','2', '3', '4', '5', '6', '7', '8']
    y = vals
    p = figure(title="Spectral Profile", x_axis_label='Spectral Bands', y_axis_label='Data Values', width=600, height=300)
    p.line(x, y, legend="Selected Point", line_color="red", line_width=2)
    p.circle(x, y, line_color="red", fill_color="white", size=8)
    p.y_range=Range1d(0, 1.0)

    show(p)
    
print('Click anywhere on the map to plot the spectral profile for that location.')
m_sprofile.on_click(get_samples)

In [ ]:
m_sprofile.clear_graphics()